In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np

class CNN(nn.Module):
    """CNN."""

    def __init__(self):
        """CNN Builder."""
        super(CNN, self).__init__()

        self.conv_layer = nn.Sequential(

            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 2
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 3
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=0.1),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
        )

        self.fc_layer = nn.Sequential(
            nn.Linear(4096, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(512, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        """Perform forward."""

        # conv layers
        x = self.conv_layer(x)

        # flatten
        x = x.view(x.size(0), -1)

        # fc layer
        x = self.fc_layer(x)

        return x

batch_size = 64

transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

cifar_train = datasets.CIFAR10(root='/CIFARDATA', train=True, download=True, transform=transform_train)
cifar_test = datasets.CIFAR10(root='/CIFARDATA', train=False, download=True, transform=transform_test)

train_loader = DataLoader(cifar_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(cifar_test, batch_size=batch_size, shuffle=False)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.shape[0], -1)

torch.manual_seed(2)
torch.cuda.manual_seed_all(2)

softmax = nn.Softmax(dim=1)

def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 2 every 20 epochs"""
    lr = learning_rate * (0.5 ** (epoch // 20))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def enable_dropout(model):
    """ Function to enable the dropout layers during test-time """
    for m in model.modules():
        if m.__class__.__name__.startswith('Dropout'):
            m.train()

def epoch(loader, model, opt=None):

    if opt:
        model.train()
    else:
        model.eval()
    
    total_loss, total_err = 0., 0.

    for X, y in loader:
        X, y = X.to(device), y.to(device)
        yp = model(X)
        loss = F.nll_loss(F.log_softmax(yp, dim=1), y)
        if opt:
            opt.zero_grad()
            loss.backward()
            opt.step()

        total_err += (yp.max(dim=1)[1] != y).sum().item()
        total_loss += loss.item() * X.shape[0]
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)

model_cnn = CNN()
model_cnn = model_cnn.to(device)

learning_rate = 0.005

opt = optim.SGD(model_cnn.parameters(), lr=learning_rate, momentum=0.9)

for t in range(50):
    adjust_learning_rate(opt, t)
    train_err, train_loss = epoch(train_loader, model_cnn, opt)
    test_err, test_loss = epoch(test_loader, model_cnn)
    print(*("{:.6f}".format(i) for i in (train_err, test_err)), sep="\t")

torch.save(model_cnn.state_dict(), "model_cnn_cifar10.pt")

model_cnn.load_state_dict(torch.load("model_cnn_cifar10.pt"))
model_cnn.eval()


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting /CIFARDATA/cifar-10-python.tar.gz to /CIFARDATA
Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


0.894100	0.837300
0.753220	0.621000
0.592920	0.526300
0.504500	0.468900
0.426080	0.369200
0.356740	0.336200
0.306420	0.273100
0.264980	0.237400
0.231980	0.232200
0.206660	0.219800
0.182880	0.198800
0.161700	0.206000
0.143480	0.201700
0.124420	0.193000
0.107640	0.199900
0.093840	0.189700
0.084580	0.200500
0.071460	0.185900
0.063780	0.181800
0.057440	0.195700
0.028220	0.169700
0.020720	0.172700
0.015980	0.171000
0.014320	0.175300
0.012760	0.170600
0.014060	0.171800
0.011500	0.169700
0.010160	0.171700
0.010240	0.175300
0.008080	0.172300
0.009660	0.175800
0.007820	0.171400
0.008140	0.174100
0.007580	0.176600
0.007740	0.178600
0.006960	0.172300
0.007640	0.169800
0.007540	0.168900
0.006380	0.171100
0.006040	0.177300
0.003660	0.169500
0.002660	0.170000
0.002400	0.168400
0.001860	0.168000
0.001800	0.168500
0.001860	0.168300
0.001620	0.170700
0.001980	0.166300
0.001840	0.167600
0.001860	0.167900


CNN(
  (conv_layer): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Dropout2d(p=0.1, inplace=False)
    (13): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layer): Sequential(
    (0): Linea